## Data download

In [1]:
!wget http://hapmap.ncbi.nlm.nih.gov/downloads/genotypes/hapmap3/plink_format/draft_2/hapmap3_r2_b36_fwd.consensus.qc.poly.map.bz2
!wget http://hapmap.ncbi.nlm.nih.gov/downloads/genotypes/hapmap3/plink_format/draft_2/hapmap3_r2_b36_fwd.consensus.qc.poly.ped.bz2

!wget http://hapmap.ncbi.nlm.nih.gov/downloads/genotypes/hapmap3/plink_format/draft_2/relationships_w_pops_121708.txt

--2015-01-12 12:01:14--  http://hapmap.ncbi.nlm.nih.gov/downloads/genotypes/hapmap3/plink_format/draft_2/hapmap3_r2_b36_fwd.consensus.qc.poly.map.bz2
Resolving hapmap.ncbi.nlm.nih.gov (hapmap.ncbi.nlm.nih.gov)... 130.14.29.113, 2607:f220:41e:4290::113
Connecting to hapmap.ncbi.nlm.nih.gov (hapmap.ncbi.nlm.nih.gov)|130.14.29.113|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10590722 (10M) [application/x-bzip2]
Saving to: ‘hapmap3_r2_b36_fwd.consensus.qc.poly.map.bz2’

100%[======================================>] 10,590,722  3.01MB/s   in 3.7s   

2015-01-12 12:01:18 (2.72 MB/s) - ‘hapmap3_r2_b36_fwd.consensus.qc.poly.map.bz2’ saved [10590722/10590722]

--2015-01-12 12:01:18--  http://hapmap.ncbi.nlm.nih.gov/downloads/genotypes/hapmap3/plink_format/draft_2/hapmap3_r2_b36_fwd.consensus.qc.poly.ped.bz2
Resolving hapmap.ncbi.nlm.nih.gov (hapmap.ncbi.nlm.nih.gov)... 130.14.29.113, 2607:f220:41e:4290::113
Connecting to hapmap.ncbi.nlm.nih.gov (hapmap.ncbi.nlm.nih.g

In [2]:
!bunzip2 hapmap3_r2_b36_fwd.consensus.qc.poly.map.bz2
!bunzip2 hapmap3_r2_b36_fwd.consensus.qc.poly.ped.bz2

# Preparation

In [11]:
import os
from collections import defaultdict

## Loading HapMap meta-data

In [12]:
f = open('relationships_w_pops_121708.txt')
pop_ind = defaultdict(list)
f.readline()  # header
offspring = []
for l in f:
    toks = l.rstrip().split('\t')
    fam_id = toks[0]
    ind_id = toks[1]
    mom = toks[2]
    dad = toks[3]
    if mom != '0' or dad != '0':
        offspring.append((fam_id, ind_id))
    pop = toks[-1]
    pop_ind[pop].append((fam_id, ind_id))
f.close()

## Sub-sampling

In [13]:
os.system('plink --recode --file hapmap3_r2_b36_fwd.consensus.qc.poly --noweb --out hapmap10 --thin 0.1 --geno 0.1')
os.system('plink --recode --file hapmap3_r2_b36_fwd.consensus.qc.poly --noweb --out hapmap1 --thin 0.01 --geno 0.1')

0

## Getting only autosomal data

In [14]:
def get_non_auto_SNPs(map_file, exclude_file):
    f = open(map_file)
    w = open(exclude_file, 'w')
    for l in f:
        toks = l.rstrip().split('\t')
        chrom = int(toks[0])
        rs = toks[1]
        if chrom > 22:
            w.write('%s\n' % rs)
    w.close()

In [15]:
get_non_auto_SNPs('hapmap10.map', 'exclude10.txt')
get_non_auto_SNPs('hapmap1.map', 'exclude1.txt')
#get_non_auto_SNPs('hapmap3_r2_b36_fwd.consensus.qc.poly.map', 'exclude.txt')

In [16]:
!plink --recode --file hapmap10 --noweb --out hapmap10_auto --exclude exclude10.txt
!plink --recode --file hapmap1 --noweb --out hapmap1_auto --exclude exclude1.txt
#geno!!!
#!plink --recode --file hapmap3_r2_b36_fwd.consensus.qc.poly --noweb --out hapmap_auto --exclude exclude.txt

PLINK v1.90b2t 64-bit (20 Dec 2014)        https://www.cog-genomics.org/plink2
(C) 2005-2014 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to hapmap10_auto.log.
Note: --noweb has no effect since no web check is implemented yet.
32095 MB RAM detected; reserving 16047 MB for main workspace.
.ped scan complete (for binary autoconversion).
Performing single-pass .bed write (144082 variants, 1184 people).
--file: hapmap10_auto-temporary.bed + hapmap10_auto-temporary.bim +
hapmap10_auto-temporary.fam written.
144082 variants loaded from .bim file.
1184 people (589 males, 595 females) loaded from .fam.
--exclude: 138890 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 988 founders and 196 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%

## Removing offspring

In [17]:
!plink --file hapmap10_auto --filter-founders --recode --out hapmap10_auto_noofs

PLINK v1.90b2t 64-bit (20 Dec 2014)        https://www.cog-genomics.org/plink2
(C) 2005-2014 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to hapmap10_auto_noofs.log.
32095 MB RAM detected; reserving 16047 MB for main workspace.
.ped scan complete (for binary autoconversion).
Performing single-pass .bed write (138890 variants, 1184 people).
--file: hapmap10_auto_noofs-temporary.bed + hapmap10_auto_noofs-temporary.bim +
hapmap10_auto_noofs-temporary.fam written.
138890 variants loaded from .bim file.
1184 people (589 males, 595 females) loaded from .fam.
196 people removed due to founder status (--filter-founders).
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 988 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35

## LD-prunning

In [18]:
!plink --file hapmap10_auto_noofs --indep-pairwise 50 10 0.1 --out keep
!plink --file hapmap10_auto_noofs --extract keep.prune.in --recode --out hapmap10_auto_noofs_ld

PLINK v1.90b2t 64-bit (20 Dec 2014)        https://www.cog-genomics.org/plink2
(C) 2005-2014 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to keep.log.
32095 MB RAM detected; reserving 16047 MB for main workspace.
.ped scan complete (for binary autoconversion).
Performing single-pass .bed write (138890 variants, 988 people).
--file: keep-temporary.bed + keep-temporary.bim + keep-temporary.fam written.
138890 variants loaded from .bim file.
988 people (488 males, 500 females) loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 988 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%40%41%42%43%44%45%46%47%48%49%50%51%52%53%54%55%

## Different coding

In [19]:
!plink --file hapmap10_auto_noofs_ld --recode12 tab --out hapmap10_auto_noofs_ld_12
!plink --make-bed --file hapmap10_auto_noofs_ld --out hapmap10_auto_noofs_ld

PLINK v1.90b2t 64-bit (20 Dec 2014)        https://www.cog-genomics.org/plink2
(C) 2005-2014 Shaun Purcell, Christopher Chang   GNU General Public License v3
Note: --recode12 flag deprecated.  Use 'recode 12 ...'.
Logging to hapmap10_auto_noofs_ld_12.log.
32095 MB RAM detected; reserving 16047 MB for main workspace.
.ped scan complete (for binary autoconversion).
Performing single-pass .bed write (55299 variants, 988 people).
--file: hapmap10_auto_noofs_ld_12-temporary.bed +
hapmap10_auto_noofs_ld_12-temporary.bim +
hapmap10_auto_noofs_ld_12-temporary.fam written.
55299 variants loaded from .bim file.
988 people (488 males, 500 females) loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 988 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%

## Single chromosome

In [1]:
!plink --recode --file hapmap10_auto_noofs --chr 2 --out hapmap10_auto_noofs_2

PLINK v1.90b3 64-bit (11 Jan 2015)         https://www.cog-genomics.org/plink2
(C) 2005-2015 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to hapmap10_auto_noofs_2.log.
7857 MB RAM detected; reserving 3928 MB for main workspace.
.ped scan complete (for binary autoconversion).
Performing single-pass .bed write (11677 variants, 988 people).
--file: hapmap10_auto_noofs_2-temporary.bed +
hapmap10_auto_noofs_2-temporary.bim + hapmap10_auto_noofs_2-temporary.fam
written.
11677 variants loaded from .bim file.
988 people (488 males, 500 females) loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 988 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%40%41%42%43%44%45